In [15]:
from spykshrk.realtime.simulator import nspike_data
from spykshrk.realtime.ripple_process import RippleFilter, RippleParameterMessage
from spykshrk.realtime.realtime_base import NoneRecordBase

import pandas as pd
pd.set_option('display.max_rows', 10)

In [2]:
%%time
bond_settings = {
    "base_dir": "/opt/data/daliu/other/mkarlsso/",
    "name": "bond",
    "days": [4],
    "epochs": [1], 
    "tetrodes": [5, 11, 12, 13, 14, 17, 19, 29]
}

anim = nspike_data.AnimalInfo(**bond_settings)

eeg = nspike_data.EEGDataStream(anim)

CPU times: user 17.3 s, sys: 596 ms, total: 17.9 s
Wall time: 17.8 s


In [16]:
eeg_df = eeg.data[4]
eeg_df

,5,11,12,13,14,17,19,29
"24,610,081.0",-117.0,102.0,81.0,125.0,-87.0,91.0,nan,nan
"24,610,087.7",-114.0,116.0,66.0,28.0,-77.0,102.0,nan,nan
"24,610,094.3",-120.0,138.0,50.0,-4.0,-83.0,115.0,nan,nan
"24,610,101.0",-141.0,142.0,53.0,17.0,-108.0,122.0,nan,nan
"24,610,107.7",-125.0,145.0,62.0,36.0,-128.0,104.0,nan,nan
...,...,...,...,...,...,...,...,...
"34,050,154.7",-48.0,-11.0,12.0,22.0,8.0,-38.0,nan,nan
"34,050,160.0",nan,nan,nan,nan,nan,nan,-19.0,-57.0
"34,050,166.7",nan,nan,nan,nan,nan,nan,-19.0,-28.0
"34,050,173.3",nan,nan,nan,nan,nan,nan,-13.0,-20.0


In [9]:

eeg_df[5].dropna().items().__next__()

(24610081.0, -117.0)

In [13]:
%%prun
rip_param_dict = { "rip_coeff1": 1.2,
                   "rip_coeff2": 0.2,
                   "ripple_threshold": 5,
                   "samp_divisor": 10000,
                   "n_above_thresh": 1,
                   "lockout_time": 7500,
                   "detect_no_ripples": False,
                   "detect_no_ripple_time": 60000,
                   "dio_gate": False,
                   "dio_gate_port": -1,
                   "enabled": True,
                   "use_custom_baseline": True,
                   "update_custom_baseline": False }

rip_param_msg = RippleParameterMessage(**rip_param_dict)

rec_base = NoneRecordBase()

rip_filt = RippleFilter(rec_base=rec_base, param=rip_param_msg, ntrode_id=5)
for eeg_times, eeg_val in eeg_df[5].dropna().items():
    rip_filt.process_data(eeg_times, eeg_val)